<a href="https://colab.research.google.com/github/DhavalRepo18/AssetOpsBench/blob/main/LLM_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Hands-on Notebook (https://tinyurl.com/ibm-llm-notebook-2025)

This notebook demonstrates how to use LangChain with a Watsonx.ai Large Language Model (LLM) and how to create and use a custom tool with LLM agents. Some key concepts:

- **LLM (Large Language Model):** A type of artificial intelligence that can understand and generate human-like text.
- **LangChain:** A popular python framework for developing applications powered by language models.
- **LLM Agent:** An entity that uses an LLM to decide which actions to take, from a set of available tools, to achieve a goal.

What we'll cover:

* Understanding what LLM agents are and why they matter
* Setting up LangChain with IBM Watsonx.ai
* Building agents that can use tools to solve real problems
* Practical examples you can implement immediately

## Install langchain with Watsonx.ai

In [1]:
!pip install -U langchain-classic langchain-ibm langchain-community langchain-core

In [31]:
import langchain
from langchain_ibm import WatsonxLLM

In [32]:
langchain.__version__

'0.1.20'

## Load Watsonx.ai credentials
Watsonx.ai: IBM's studio for training, validating, tuning, and deploying foundation models (LLMs). This notebook uses watsonx.ai to access to various LLMs, e.g. Llama, Granite, Ministral.

In [33]:
import os
from getpass import getpass

watsonx_api_key = getpass()
os.environ["WATSONX_APIKEY"] = watsonx_api_key

··········


In [34]:
watsonx_project_id = getpass()
os.environ["WATSONX_PROJECT_ID"] = watsonx_project_id

··········


In [35]:
os.environ["WATSONX_URL"] = "https://us-south.ml.cloud.ibm.com"

In [36]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

## Initiate a LLM call

In [42]:
# Model parameter configurations
parameters = {
    "decoding_method": "greedy",  # or "sample" for creative outputs
    "max_new_tokens": 1000,
    "min_new_tokens": 1,
    "temperature": 0.0, # Controls randomness (0-2)
    "top_k": 50,  # Limits vocabulary to top k tokens
    "top_p": 1, # Nucleus sampling threshold
}

We use Llama-4-Maverick-17B-128E-Instruct-FP8, Meta's latest open-source AI model released in April 2025.

In [43]:
watsonx_llm = WatsonxLLM(
    model_id="meta-llama/llama-4-maverick-17b-128e-instruct-fp8",
    url=os.environ["WATSONX_URL"],
    project_id=os.environ["WATSONX_PROJECT_ID"],
    params=parameters,
)

In [56]:
response = watsonx_llm.invoke("What is the square root of 144?")
print(response)

 | HIX Tutor
The square root of 144 is 12.
#sqrt(144) = 12#
#12^2 = 144#
#(-12)^2 = 144#
#sqrt(144) = +-12#
However, by convention, #sqrt(x)# is used to represent the principal square root of #x#, that is the non-negative one. So:
#sqrt(144) = 12#
- How do you simplify #sqrt(32xy^2) div sqrt4#?
How do you simplify #sqrt(32x^4)#? 
How do you simplify #sqrt(2/3)*sqrt(3/8)#?
How do you simplify #sqrt(32a^8b)+sqrt(50a^16b)#? 
How do you simplify #sqrt(9x^2) /sqrt(18y^2)#? <|end_of_text|>


## Create a LLM (ReAct) agent with a simple custom tool
A Large Language Model (LLM) agent is an intelligent system that uses an LLM not just for generating text, but also for deciding when and how to use external tools. Instead of trying to solve every problem directly, the agent interprets a user’s request, chooses the right tool (like a calculator, database, or API), and combines the results into a useful response. This makes LLM agents more reliable and practical than standalone models, since they can extend their capabilities with specialized functions.

In [47]:
# Instead of langchain.agents, use langchain_classic.agents
from langchain_classic.agents import initialize_agent, AgentType
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain.tools import Tool

# The rest of your code (math, re, initialize_agent) remains the same!
import re
import math

Currently, one general consensus is that LLMs struggle with math problems. One effective way to address this is by creating an LLM agent that can use external tools to perform calculations. Here, we define a simple square root function and integrate it as a tool for the agent. This allows the LLM to delegate precise mathematical operations to the tool, improving accuracy and reliability.

In [15]:
def square_root(number: float) -> str:
    """Calculate the square root of a number."""
    if number < 0:
        return "Error: Cannot calculate square root of a negative number."
    result = math.sqrt(number)
    return f"The square root of {number} is {result}"

# Create the square root tool using lambda with regex
sqrt_tool = Tool(
    name="SquareRoot",
    func=lambda s: square_root(float(re.findall(r'-?\d+\.?\d*', s)[0])),
    description="Calculate the square root of a number. Input should be a single number."
)

In [61]:
tools = [sqrt_tool]
agent = initialize_agent(
    tools=tools,
    llm=watsonx_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

**ReAct** (Reason+Act) prompting is a technique that combines **reasoning** and **acting** to enable AI agents to solve complex tasks through an iterative cycle. The approach works through three repeating steps: **Thought** (the agent reasons about what it knows and what to do next), **Action** (it executes a concrete step like searching the web or calling a tool), and **Observation** (it receives and processes the results). This thought-action-observation loop continues until the agent gathers enough information to complete the task.

Using ReAct-style agents combines the LLM’s reasoning abilities with external tool usage. The agent doesn’t just blindly answer a question — it thinks step by step, decides which tool (or knowledge source) to use, executes the action, observes the result, and continues reasoning if necessary. This approach is particularly powerful for complex tasks, such as multi-step math, data queries, or planning, where the model alone might make errors.

In [55]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

SquareRoot(s) - Calculate the square root of a number. Input should be a single number.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [SquareRoot]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [60]:
result = agent.run("What are the factors of 156?")



> Entering new AgentExecutor chain...
 To find the factors of 156, I should first determine its prime factorization. I can start by checking if 156 is divisible by the smallest prime numbers.

Action: SquareRoot
Action Input: 156
Observation:
Observation: The square root of 156.0 is 12.489995996796797
Thought: The square root of 156 is approximately 12.49. This means I should check prime numbers up to 12 to find the prime factors of 156.

Thought: I will start checking divisibility by the smallest prime numbers: 2, 3, 5, 7, 11.

Thought: 156 is even, so it is divisible by 2.

Thought: 156 / 2 = 78. Continuing, 78 / 2 = 39. So, 156 = 2^2 * 39.

Thought: Now, I need to factorize 39. It is divisible by 3 because 39 / 3 = 13. So, 39 = 3 * 13.

Thought: Therefore, the prime factorization of 156 is 2^2 * 3 * 13.

Thought: To find all the factors, I will use the prime factorization. The factors will be all possible combinations of the prime factors 2, 3, and 13.

Thought: The prime factors 

In [21]:
print("Answer:", result)

Answer: 12.0


## Create a LLM (ReAct) agent with pre-built Langchain tool
All pre-built langchain tools can be found [here](https://python.langchain.com/docs/integrations/tools/).

Some examplar tools are Google search, Math, Yahoo Finance.

The following question demonstrates thought-action-observation loop more clearly.

In [23]:
result = agent.run("What is 2 times 6 plus 100 minus 1?")



> Entering new AgentExecutor chain...
 To solve this problem, I need to follow the order of operations (PEMDAS/BODMAS). First, I will calculate 2 times 6, then add 100, and finally subtract 1.

Action: Calculator
Action Input: 2 * 6
Observation:

ValueError: unknown format from LLM: The result of 2 * 6 is 12.
```text
2 * 6
```
...numexpr.evaluate("2 * 6")...
```output

In [24]:
print("Answer:", result)

Answer: 12.0


Note that the built-in llm-math tool in LangChain is known to be a bit error-prone. This is mostly due to string parsing issues. LLM could accidentally include extra spaces, units, or explanation in the math expression, the evaluator can fail.  That's why many people now prefer structured tools or libraries like SymPy/NumPy wrapped as tools.

## Create a LLM (React) agent with multiple tools

In [62]:
!pip install -U ddgs wikipedia

In [63]:
from langchain_community.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# ==================== TOOL 1: DuckDuckGo Search ====================
search = DuckDuckGoSearchRun()

# ==================== TOOL 2: Wikipedia ====================
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# ==================== CREATE TOOLS LIST ====================
tools = [
    Tool(
        name="WebSearch",
        func=search.run,
        description="Search the internet for all information."
    ),
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="Search Wikipedia for detailed historical information, biographies, scientific facts, or encyclopedic knowledge. Good for established facts and background information."
    )
]

In [64]:
agent = initialize_agent(
    tools=tools,
    llm=watsonx_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

In [65]:
result = agent.run("who is the director of Avatar? Provide a one-paragraph biography.")



> Entering new AgentExecutor chain...
 To answer this question, I need to identify the director of the movie "Avatar" and then provide a brief biography of that person. The first step is to confirm the director's name.

Action: Wikipedia
Action Input: director of Avatar
Observation:
Observation: Page: List of Avatar: The Last Airbender characters
Summary: This is a list of significant characters from the Nickelodeon animated television series Avatar: The Last Airbender and its sequel The Legend of Korra, co-created by Bryan Konietzko and Michael Dante DiMartino, as well the live-action Avatar series. This list also includes characters from the comic continuations as well as the Kyoshi, Yangchen and Roku prequel novels by FC Yee, Michael Dante DiMartino and Randy Ribay.



Page: Mako (The Legend of Korra)
Summary: Mako (Chinese: 馬高; pinyin: Mǎ Gāo) is a major character in Nickelodeon's animated television series The Legend of Korra, which aired from 2012 to 2014. He is part of the Ava

In [30]:
print("Answer:", result)

Answer: James Francis Cameron, born on August 16, 1954, is a renowned Canadian filmmaker and deep-sea explorer. His films are known for combining cutting-edge technology with classical techniques and have grossed over $10 billion worldwide, making him the second-highest-grossing film director of all time. Cameron's career in filmmaking began with the 1978 science fiction short "Xenogenesis," which he directed, wrote, and produced. He has since become a prominent figure in the industry, known for directing films like "Avatar" (2009) and "Titanic" (1997). Cameron's work has been recognized for its innovative use of technology and its impact on the film industry. In addition to his filmmaking achievements, Cameron is also known for his deep-sea exploration endeavors. His contributions to filmmaking and exploration have cemented his status as a significant figure in both industries.
